In [ ]:
!pip install mediapipe

In [ ]:

# Computer Vision features
import cv2
from google.colab.patches import cv2_imshow
from keras.preprocessing import image
from keras.utils import np_utils
import mediapipe as mp

# Data processing
import math
import numpy as np
import pandas as pd

# Data visualization
import matplotlib.pyplot as plt

# File path & temporal processes
import os
import time

In [ ]:
# Set the holistic model
mp_holistic = mp.solutions.holistic

# Set the drawing utilities
mp_drawing = mp.solutions.drawing_utils

In [ ]:
def mediapipe_detection(image, model):
    
    # Converts the image color.
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    
    # Prevents image write
    image.flags.writeable = False                  
    
    # Makes the prediction.
    results = model.process(image)
    
    # Enables image write.
    image.flags.writeable = True
    
    # Convert back to BGR.
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    
    # Return the image and prediction results.
    return image,results

In [ ]:
# draw_landmarks: Takes a frame and results then applies the landmark visualizations to
# hand and pose.
def draw_landmarks(image, results):
    
    # Draw left hand points.                    
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, 
                              mp_holistic.HAND_CONNECTIONS)
    # Draw right hand points.
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, 
                              mp_holistic.HAND_CONNECTIONS) 
    
    # Draw pose points.
    mp_drawing.draw_landmarks(image, results.pose_landmarks, 
                              mp_holistic.POSE_CONNECTIONS)

In [ ]:
# extract_keypoints: Gets the x,y,z coordinates of the keypoints of a frame and returns a concatenated
# array of those coordinates for the pose, left, and right hand.
def extract_keypoints(results):

    # Gets and flattens the coordinates for each of the landmark areas. If there
    # are no values for the frame, 0's are returned.
    pose = np.array([[res.x, res.y, res.z, res.visibility] for 
                     res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for 
                   res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for 
                   res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    
    # Returns the concatenated np array for each of the landmarks.
    return np.concatenate([pose, lh, rh])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Set the folder path for the numpy arrays.
DATA_PATH = os.path.join('Data')

# Get the names of the gestures from the dataset dataframe.
# ********EXTRACT THE GESTURE NAME FROM THE DATASET HERE*******
# gestures = np.array(getGestureNameList(dataFrame, columnNumber))
gestures = np.array(['thumbsup'])

# Set the number of videos contained in the dataset.
# ********EXTRACT THE NUMBER OF ROWS FROM THE DATASET HERE*******
no_sequences = 4

Create folders for each of the gestures. ****THIS NEEDS WORK****

In [ ]:
# Loops through each of the gestures to make a folder for the main action
# and subfolders for each video of the action.
for gesture in gestures:
  dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, gesture))).astype(int))

  # Make a subfolder for each video within the gesture. If the folder already
  # exists, pass.
  for sequence in range(1,no_sequences+1):
    try:
      os.makedirs(os.path.join(DATA_PATH,action,str(dirmax+sequence)))
    except:
      pass

FileNotFoundError: ignored

Takes the video, performs keypoints extractions by frame, and saves the resulting numpy arrays to a folder. **** The numpy_array_path needs some tweaking****

In [ ]:
# extractFrames: Performs keypoints extraction on each frame of the input video
# and saves the keypoints to a numpy array folder
def extractFrames(videoMP4,action):
  with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # Sets count.
    count = 0
    
    # Sets the videoFile name. **INCLUDE** the extension
    videoFile = videoMP4

    # Captures the video
    cap = cv2.VideoCapture(videoMP4)

    # Set the framerate 
    frameRate = cap.get(5)

    # While the video is running, read in the video frames
    # and write them to a file. 
    while(cap.isOpened()):

      # Sets the frame number
      frameId = cap.get(1)

      # Reads in the frame.
      ret, frame = cap.read()

      # Display the video frame with landmarks overlaid.
      #cv2_imshow(frame)

      # If there are no more frames, the capturing stops.
      # Otherwise, the next frame is read in as a jpg 
      # to the file.
      if (ret != True):
          break

      # If the frame number is divisible by 4, extract
      # and write the keypoints to a subfolder.
      if (frameId % 4 == 0):

        # Keypoints detections.
        image,results = mediapipe_detection(frame,holistic)

        #filename ="frame%d.jpg" % count;
        count+=1
        #cv2.imwrite(filename, frame)
        #print(filename)

        #---Export keypoints---
        # Get the keypoints array
        keypoints = extract_keypoints(results)

        #***** Set the numpy array path(Data/Gesture/GestureVideo#/numpyfile)
        numpy_array_path = os.path.join("Data",action,str(count))

        # Save the keypoints to the path.
        np.save(numpy_array_path,keypoints)

    # Stops the capture.
    cap.release()
    print ("Frame Capture Finished.")

Video keypoint Extraction

In [ ]:
videoGestures = ['thumbsup01.mp4','thumbsup02.mp4','thumbsup03.mp4',
             'thumbsup04.mp4']

#for vid in videoGestures:
#extractFrames('vid')

extractFrames('thumbsup01.mp4','thumbsup')

FileNotFoundError: ignored